<a href="https://colab.research.google.com/github/Jeongmin0658/kentech_EF_CCP/blob/main/codes/particle_based/Image_data/Image_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import torch
# if torch.backends.mps.is_available():
#     mps_device = torch.device("mps")
#     x = torch.ones(1, device=mps_device)
#     print (x)
# else:
#     print ("MPS device not found.")

# x = torch.rand(5, 3)
# print(x)

# # Check if CUDA is available, and set PyTorch to use GPU or CPU accordingly
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
# print(torch.cuda.is_available())

In [ ]:
import numpy as np
import umap.umap_ as umap
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns

class ImageLoad:

    def __init__(self):
        self.all_images = []  # List to store all the image grids

    @staticmethod
    def read_image_and_info(image, file, delimiter='\t'):
        # Load the data
        loaded_data = np.loadtxt(file, delimiter=delimiter)

        # Extract columns
        r = loaded_data[:, 0]  # radius
        m = loaded_data[:, 1]  # mass
        dm = loaded_data[:, 2]  # differential mass
        ndm = loaded_data[:, 3]  # normalized differential mass

        # Load grid
        grid = np.loadtxt(image, dtype=int)

        return grid, r, m, dm, ndm

    @staticmethod
    def extract_feature_vector(r, m, f1, f2):
#         # If the grid is 2D, flatten it
#         if len(grid.shape) == 2:
#             print ("Matrix fed?")
#             grid = grid.flatten()

        # Concatenate the vectors
        return np.concatenate([m, f1, f2])

    def cluster_images(self, p, nimage, snap_every):
        # feature vectors
        all_feature_vectors = []

        # EDEN
        p = 0
        for n in range(nimage):
            for j, i in enumerate(snap_every):
                grid, r, m, f1, f2 = self.read_image_and_info(
                    image='images/smooth/image_p%d_#%d_%d.txt' % (p, n, i),
                    file='images/smooth/info_p%d_#%d_%d.txt' % (p, n, i),
                    delimiter='\t')
                self.all_images.append(grid)  # Append the image grid to the all_images list
                feature_vector = self.extract_feature_vector(r, m, f1, f2)
                all_feature_vectors.append(feature_vector)

        # DLA
        p = 1
        for n in range(nimage):
            for j, i in enumerate(snap_every):
                grid, r, m, f1, f2 = self.read_image_and_info(
                    image='images/dendritic/image_p%d_#%d_%d.txt' % (p, n, i),
                    file='images/dendritic/info_p%d_#%d_%d.txt' % (p, n, i),
                    delimiter='\t')
                self.all_images.append(grid)  # Append the image grid to the all_images list
                feature_vector = self.extract_feature_vector(r, m, f1, f2)
                all_feature_vectors.append(feature_vector)

        # feature vector
        self.all_feature_vectors = np.vstack(all_feature_vectors)

    # Function to get all images
    def get_all_images(self):
        return self.all_images

    # Function to get all images
    def get_all_features(self):
        return self.all_feature_vectors

In [ ]:
####
# Run simulation
# Outputs saved in the folder of "images/dendritic"
# XXX: sticking probability p
# YYY: image index
# ZZZ: number of deposits
# Images: image_pXXX_#YYY_ZZZ.txt
# Their characterization info: info_pXXX_#YYY_ZZZ.txt
####
p=0 # sticking prob.
nimage=512 # number of images
#
size_image=200 # square image size
ndeposit=1000 # number of deposits
snap_every=[100, 200, 400, 600, 800,1000]

# Define my method
image_loader = ImageLoad()
image_loader.cluster_images(p, nimage, snap_every)
all_images = image_loader.get_all_images()
all_feature_vectors = image_loader.get_all_features()